In [1]:
import pandas as pd
import numpy as np
import networkx as nx 
from numpy.linalg import norm

In [2]:
# # from colab run this 
# !git clone https://github.com/gru13/lncRNA_Disease_association.git
# df_lnc_die = pd.read_csv(r"./lncRNA_Disease_association/datasets/lncrnaDiesease3/website_alldata_p1.csv")
# df_lnc_mir = pd.read_csv(r"./lncRNA_Disease_association/datasets/LncRNASNP/Homo_sapiens_lncRNA_miRNA_interactions.csv")
# df_mir_die = pd.read_csv(r"./lncRNA_Disease_association/datasets/rnaDisease/RNADisease_RNA-disease_miRNA_predict.csv")

In [3]:
# from local system

df_lnc_die = pd.read_csv(r"./lncrnaDiesease3/website_alldata_p1.csv")
df_lnc_mir = pd.read_csv(r"./LncRNASNP/Homo_sapiens_lncRNA_miRNA_interactions.csv")
df_mir_die = pd.read_csv(r"./rnaDisease/RNADisease_RNA-disease_miRNA_predict.csv")


In [4]:

df_lnc_die.head(),df_lnc_mir.head(),df_mir_die.head()

(        lncRNA                    Disease  PubMed ID
 0  ARHGAP5-AS1  Carcinoma, Hepatocellular   36354136
 1       HOTTIP               Osteosarcoma   33475442
 2       HOTTIP                     Glioma   35402278
 3       HOTTIP             Retinoblastoma   33784880
 4       HOTTIP          stomach carcinoma   32633359,
             lncRNA           miRNA
 0  NONHSAT000002.2  hsa-miR-15a-5p
 1  NONHSAT000002.2  hsa-miR-15a-3p
 2  NONHSAT000002.2   hsa-miR-16-5p
 3  NONHSAT000002.2   hsa-miR-21-3p
 4  NONHSAT000002.2   hsa-miR-28-5p,
           miRNA                    Disease
 0  hsa-let-7a-1  Abdominal aortic aneurysm
 1  hsa-let-7a-2  Abdominal aortic aneurysm
 2  hsa-let-7a-3  Abdominal aortic aneurysm
 3    hsa-let-7c  Abdominal aortic aneurysm
 4    hsa-let-7d  Abdominal aortic aneurysm)

In [5]:
dataset1 = pd.merge(df_lnc_mir,df_lnc_die,on=['lncRNA'],how='inner')
dataset2 = pd.merge(df_lnc_mir,df_mir_die,on=['miRNA'],how='inner')

In [6]:
dataset1

,lncRNA,miRNA,Disease,PubMed ID
0,NONHSAT000612.2,hsa-miR-570-3p,Osteoarthritis,34780784
1,NONHSAT000612.2,hsa-miR-651-3p,Osteoarthritis,34780784
2,NONHSAT000612.2,hsa-miR-449b-3p,Osteoarthritis,34780784
3,NONHSAT000612.2,hsa-miR-1468-3p,Osteoarthritis,34780784
4,NONHSAT000612.2,hsa-miR-874-5p,Osteoarthritis,34780784
...,...,...,...,...
590,NONHSAT015292.2,hsa-miR-9902,Osteoarthritis,34780784
591,NONHSAT015292.2,hsa-miR-10522-5p,Osteoarthritis,34780784
592,NONHSAT015292.2,hsa-miR-10526-3p,Osteoarthritis,34780784
593,NONHSAT015292.2,hsa-miR-9851-3p,Osteoarthritis,34780784


In [7]:
dataset2

,lncRNA,miRNA,Disease
0,NONHSAT000002.2,hsa-miR-15a-5p,Abortion habitual
1,NONHSAT000002.2,hsa-miR-15a-5p,Acoustic neuroma
2,NONHSAT000002.2,hsa-miR-15a-5p,Acquired immunodeficiency syndrome
3,NONHSAT000002.2,hsa-miR-15a-5p,Acute coronary syndrome
4,NONHSAT000002.2,hsa-miR-15a-5p,Acute kidney failure
...,...,...,...
9376652,NONHSAT017163.2,hsa-miR-523-3p,Intellectual disability
9376653,NONHSAT017163.2,hsa-miR-523-3p,Leukemia
9376654,NONHSAT017163.2,hsa-miR-523-3p,Neurodegenerative diseases
9376655,NONHSAT017163.2,hsa-miR-523-3p,Skin disease


In [8]:
df = dataset2.copy()
df

,lncRNA,miRNA,Disease
0,NONHSAT000002.2,hsa-miR-15a-5p,Abortion habitual
1,NONHSAT000002.2,hsa-miR-15a-5p,Acoustic neuroma
2,NONHSAT000002.2,hsa-miR-15a-5p,Acquired immunodeficiency syndrome
3,NONHSAT000002.2,hsa-miR-15a-5p,Acute coronary syndrome
4,NONHSAT000002.2,hsa-miR-15a-5p,Acute kidney failure
...,...,...,...
9376652,NONHSAT017163.2,hsa-miR-523-3p,Intellectual disability
9376653,NONHSAT017163.2,hsa-miR-523-3p,Leukemia
9376654,NONHSAT017163.2,hsa-miR-523-3p,Neurodegenerative diseases
9376655,NONHSAT017163.2,hsa-miR-523-3p,Skin disease


In [9]:
df = dataset2.copy()
disease = df['Disease'].unique()
lncRNA = df['lncRNA'].unique()
miRNA = df['miRNA'].unique()

In [10]:
# len(df['lncRNA'].unique()),len(df['miRNA'].unique()),len(df['Disease'].unique()),

CReation of the graph that contains the totaly connected with miRNA, lncRNA, Disease

In [11]:
LM_Graph = nx.Graph()
MD_Graph = nx.Graph()
LD_Graph = nx.Graph()

LM_Graph.add_nodes_from(lncRNA)
LM_Graph.add_nodes_from(miRNA)

LD_Graph.add_nodes_from(lncRNA)
LD_Graph.add_nodes_from(disease)

MD_Graph.add_nodes_from(miRNA)
MD_Graph.add_nodes_from(disease)

In [12]:
for a in df.values:
    LM_Graph.add_edge(a[0],a[1])
    MD_Graph.add_edge(a[1],a[2])
    LD_Graph.add_edge(a[0],a[2])

CReation of the Lncrna-disease , Lncrna-miRNA, miRNA-Disease assoction matrix (LM,LD,MD)

In [13]:
LM_index = {a:b for (a,b) in zip(LM_Graph.nodes(),range(len(LM_Graph.nodes())))}
MD_index = {a:b for (a,b) in zip(MD_Graph.nodes(),range(len(MD_Graph.nodes())))}
LD_index = {a:b for (a,b) in zip(LD_Graph.nodes(),range(len(LD_Graph.nodes())))}

In [14]:
LM_M_A = nx.adjacency_matrix(LM_Graph).todense()
MD_M_A = nx.adjacency_matrix(MD_Graph).todense()
LD_M_A = nx.adjacency_matrix(LD_Graph).todense()

In [15]:
LM_M = np.zeros(shape=(len(lncRNA),len(miRNA)))
MD_M = np.zeros(shape=(len(miRNA), len(disease)))
LD_M = np.zeros(shape=(len(lncRNA), len(disease)))

In [16]:
for a in range(len(lncRNA)):
    for b in range(len(miRNA)):
        LM_M[a][b] = LM_M_A[LM_index[lncRNA[a]]][LM_index[miRNA[b]]]
LM_M

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
for a in range(len(miRNA)):
    for b in range(len(disease)):
        MD_M[a][b] = MD_M_A[MD_index[miRNA[a]]][MD_index[disease[b]]]
MD_M

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
import numpy as np

def cosine_similarity(matrix):
  """
  Calculates the cosine similarity between rows or columns of a matrix.

  Args:
      matrix (numpy.ndarray): A 2D array representing the data.

  Returns:
      numpy.ndarray: A 2D array containing the cosine similarity between rows (if axis=1) 
                     or columns (if axis=0) of the input matrix.
  """
  # Calculate row-wise and column-wise norms
  row_norms = np.linalg.norm(matrix, axis=1)
  col_norms = np.linalg.norm(matrix, axis=0)

  # Avoid division by zero
  col_norms[col_norms == 0] = 1e-8  # Set a small value to prevent division by zero

  # Calculate cosine similarity
  similarity = np.divide(matrix, np.outer(row_norms, col_norms))
  return similarity

LD = LM_M.dot(MD_M)
CD = cosine_similarity(LD)
CL = cosine_similarity(LD.T)
CD,CL

(array([[7.09917341e-05, 7.02685899e-05, 8.50972397e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [7.30268427e-05, 1.00393011e-04, 1.21578762e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.36088037e-04, 1.44323359e-04, 1.35939727e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 array([[7.09917341e-05, 7.30268427e-05, 1.36088037e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [7.02685899e-05, 1.00393011e-04, 1.44323359e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [8.50972397e-05, 

In [19]:
sl = np.zeros(shape=(len(lncRNA),len(lncRNA)))
SD = np.zeros(shape=(len(disease),len(disease))) 

In [20]:
sl.shape

(7554, 7554)

In [21]:
lam = 0.2
for i in range(len(lncRNA)):
    for j in range(len(lncRNA)):
        sl[i][j] = (lam*norm(LD_M[i]*LD_M[j])+(1-lam)*norm(LD_M[i]-LD_M[j]))/np.sqrt(len(lncRNA))
sl

In [ ]:
LM